# Data Wrangling Proyecto Constituyentes

La extracción de los datos de desde el servel se puede encontrar <a href="https://github.com/javosch/servel_scraping">aqui</a>.<br>
Los datos de los distritos fueron extraidos a través de <a href="https://wikitable2csv.ggor.de/">WikiTables to CSV</a> del sitio de <a href="https://es.wikipedia.org/wiki/Divisi%C3%B3n_electoral_de_Chile">Division Electoral de Chile</a> en Wikipedia que a su vez está reespaldado por las respectivas notas al pie de página.<br>
<br>
*Aqui no se incluyen los 17 escaños para pueblos originarios: 7 para el pueblo mapuche, 2 para el pueblo aimara y uno para cada uno de los pueblos restantes; quechua, atacameño, colla, diaguita, kawésqar, yagán, chango y rapanui.

In [21]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 15)

In [2]:
path = '/home/anonimo/Documentos/servel/'
file1 = 'servel'
file2 = 'distritos_constituyente'

### Normalizar ambas tablas

Quitar acentos y aplicar mayúsculas a las columnas de comunas. Ademas reemplazar aquellos errores que se encuentren.

In [3]:
df_distritos = pd.read_csv(path+file2+'.csv')
df_servel = pd.read_csv(path+file1+'.csv')
df_distritos['Comunas'] = df_distritos['Comunas'].apply(lambda x: x.upper())
df_distritos['Comunas'] = df_distritos['Comunas'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df_servel.columns = ['Comunas'] + list(df_servel.columns[1:])
df_servel['Comunas'] = df_servel['Comunas'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [4]:
servel_comunas = list(df_servel['Comunas'])

In [5]:
faltantes = list(df_distritos[~df_distritos['Comunas'].isin(servel_comunas)]['Comunas'])
faltantes

['LA CALERA', 'TREGUACO', "O'HIGGINS", 'CABO DE HORNOS']

Quedan 4 sin reemplazar. Reemplazo manual...

Según [Municipalidad de Trehuaco](#https://www.trehuaco.com/Wordpress/) es con "H" y no con "G".

In [6]:
cambios_servel = ['CALERA', 'CABO DE HORNOS(EX-NAVARINO)']
df_servel['Comunas'] = df_servel['Comunas'].replace(cambios_servel, ['LA CALERA', 'CABO DE HORNOS'])

In [7]:
cambios_distritos = ['TREGUACO', "O'HIGGINS"]
df_distritos['Comunas'] = df_distritos['Comunas'].replace(cambios_distritos, ['TREHUACO', 'OHIGGINS'])

Comprobación de que hayan sido corregidos

In [8]:
correcciones_servel = ['LA CALERA', 'CABO DE HORNOS']
df_servel[df_servel['Comunas'].isin(correcciones_servel)]

,Comunas,Total Mesas,Total Electores,Total Votación,% Participación
46,LA CALERA,131.0,44.986,24.125,"53,63"
336,CABO DE HORNOS,5.0,1.916,645.000,"33,66"


In [9]:
correcciones_distritos = ['TREHUACO', 'OHIGGINS']
df_distritos[df_distritos['Comunas'].isin(correcciones_distritos)]

,Región,Distrito constituyente,Comunas
209,Ñuble,195 convencionales constituyentes,TREHUACO
333,Aysén,273 convencionales constituyentes,OHIGGINS


Chequeo Final

In [10]:
servel_comunas = list(df_servel['Comunas'])
faltantes = list(df_distritos[~df_distritos['Comunas'].isin(servel_comunas)]['Comunas'])
faltantes

[]

Corregir el nombre de "Metropolitanade Santiago" a "Metropolitana de Santiago"

In [11]:
df_distritos['Región'] = df_distritos['Región'].replace(['Metropolitanade Santiago'], 'Metropolitana de Santiago')
df_distritos['Región'] = df_distritos['Región'].replace(['Magallanes y laAntártica Chilena'], 'Magallanes y la Antártica Chilena')

#### Separar las datos del distrito y separar el distrito de la cantidad de constituyentes

In [12]:
df_distritos['Distrito constituyente'] = [element[0] for element in list(df_distritos['Distrito constituyente'].str.split(' '))]
df_distritos['Constituyentes'] = [element[-1] for element in df_distritos['Distrito constituyente']]
df_distritos['Distrito'] = [element[0] if (len(element) == 2) else element[:2] for element in df_distritos['Distrito constituyente']]
df_distritos.drop('Distrito constituyente', axis=1, inplace=True)

### Unir Tablas

In [18]:
df = df_distritos.merge(df_servel, on='Comunas', how='left', right_index=False)
df.drop(['% Participación', 'Total Votación'], axis=1, inplace=True)

La tabla va quedando así

In [22]:
df

,Región,Comunas,Constituyentes,Distrito,Total Mesas,Total Electores
0,Arica y Parinacota,ARICA,3,1,522.0,180.315
1,Arica y Parinacota,CAMARONES,3,1,10.0,3.471
2,Arica y Parinacota,GENERAL LAGOS,3,1,6.0,2.158
3,Arica y Parinacota,PUTRE,3,1,14.0,5.554
4,Tarapacá,ALTO HOSPICIO,3,2,171.0,57.742
...,...,...,...,...,...,...
341,Magallanes y laAntártica Chilena,PUNTA ARENAS,3,28,397.0,122.555
342,Magallanes y laAntártica Chilena,RIO VERDE,3,28,2.0,771.000
343,Magallanes y laAntártica Chilena,SAN GREGORIO,3,28,3.0,901.000
344,Magallanes y laAntártica Chilena,TIMAUKEL,3,28,2.0,829.000


___________________________________________________________

# Data Wrangling Geomap

Los datos para poder usar el mapa vectorial fueron descargados desde <a href="https://www.bcn.cl/siit/mapas_vectoriales">Biblioteca del Congreso Nacional de Chile</a>.

In [15]:
import geopandas as gpd

In [25]:
path = '/home/anonimo/Documentos/servel/geomap/'
file_name = 'comunas.dbf'

tabla = gpd.read_file(path+file_name)
tabla_geomap = pd.DataFrame(tabla)

In [24]:
df[df['Comunas'].str.startswith('MAR')]

,Región,Comunas,Constituyentes,Distrito,Total Mesas,Total Electores
12,Antofagasta,MARIA ELENA,4,3,17.0,6.451
128,Metropolitana de Santiago,MARIA PINTO,5,14,33.0,11.262
157,O'Higgins,MARCHIGUE,4,16,22.0,6.787
286,Los Ríos,MARIQUINA,4,24,61.0,18.622


In [26]:
tabla_geomap

,objectid,shape_leng,dis_elec,cir_sena,cod_comuna,codregion,st_area_sh,st_length_,Region,Comuna,Provincia,geometry
0,48,170038.624165,16,8,6204,6,9.685774e+08,206184.271675,Región del Libertador Bernardo O'Higgins,Marchigüe,Cardenal Caro,"POLYGON ((-7992818.789 -4056669.319, -7992731...."
1,29,125730.104795,15,8,6102,6,4.157446e+08,151911.576827,Región del Libertador Bernardo O'Higgins,Codegua,Cachapoal,"POLYGON ((-7831651.813 -4022443.109, -7831596...."
2,30,63026.084422,15,8,6103,6,1.448565e+08,76355.326122,Región del Libertador Bernardo O'Higgins,Coinco,Cachapoal,"POLYGON ((-7892616.357 -4060673.364, -7892621...."
3,31,89840.903562,15,8,6104,6,3.256572e+08,108874.623150,Región del Libertador Bernardo O'Higgins,Coltauco,Cachapoal,"POLYGON ((-7906457.717 -4051723.734, -7906266...."
4,78,122626.493264,23,11,9121,9,6.990727e+08,156680.410681,Región de La Araucanía,Cholchol,Cautín,"POLYGON ((-8121756.186 -4645860.295, -8121644...."
...,...,...,...,...,...,...,...,...,...,...,...,...
341,255,130240.555805,19,10,16305,16,8.751006e+08,160459.137929,Región de Ñuble,San Nicolás,Punilla,"POLYGON ((-8041950.854 -4349201.562, -8041735...."
342,253,311061.932415,19,10,16304,16,2.393007e+09,388109.613857,Región de Ñuble,San Fabián,Punilla,"POLYGON ((-7931328.549 -4363544.179, -7931320...."
343,245,159257.906431,19,10,16303,16,7.638173e+08,195593.933727,Región de Ñuble,Ñiquén,Punilla,"POLYGON ((-8026032.185 -4319505.012, -8025933...."
344,243,157467.049667,19,10,16104,16,1.044405e+09,196962.664313,Región de Ñuble,El Carmen,Ñuble,"POLYGON ((-7986315.104 -4415272.935, -7986309...."
